## 그룹연산
- 데이터를 집계하거나 변환하는 등의 작업을 한번에 처리
- 분할-반영-결합 => (split-apply-combine)

## 11-1 데이터 집계하기 -groupby 메서드

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# 데이터 불러오기
df = pd.read_csv('data/gapminder.tsv', sep='\t')
df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [4]:
# year 열을 기준으로 데이터를 그룹화한 다음 lifeExp 열의 평균 구하기
avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


### 분할-반영-결합 과정 살펴보기

In [5]:
# year 열의 데이터를 중복 없이 추출 => '분할'
# groupby 메서드에 열 이름을 전달하면 '분할' 작업이 먼저 일어남
years = df['year'].unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [6]:
# 연도별로 데이터를 추출 => '반영'
y1952 = df.loc[df['year'] == 1952]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [7]:
# lifeExp 열의 평균값 구하기 => '반영'
y1952_mean = y1952['lifeExp'].mean()
print(y1952_mean)

49.05761971830987


In [8]:
# 반영 부분을 반복하여 마지막에 데이터프레임 형태로'결합'을 함

### groupby 메서드와 함께 사용하는 집계 메서드
- count : 누락값을 제외한 데이터 수를 반환
- size  : 누락값을 포함한 데이터 수를 반환
- mean  : 평균값 반환
- std   : 표준편차 반환
- max   : 최댓값 반환
- min   : 최솟값 반환
- sum   : 전체 합 반환
- var   : 분산 반환
- sem   : 평균의 표준편차 반환
- describe : 데이터 수, 평균, 표준 편차, 최소값, 백분위수(25,50,75), 최댓값을 모두 반환
- first : 첫번째 행 반환
- last  : 마지막 행 반환
- nth   : n번째 행 반환
- quantile(q=?) : 백분위수 (?)% 

### agg 메서드로 사용자 함수와 groupby 메서드 조합하기

In [9]:
def me(a):
    print(a)

In [10]:
# 함수 적용시 컬럼의 값을 시리즈 타입으로 각 행씩 출력
# df['lifeExp'].apply(me)

28.801
30.331999999999997
31.997
34.02
36.088
38.438
39.854
40.821999999999996
41.674
41.763000000000005
42.129
43.828
55.23
59.28
64.82
66.22
67.69
68.93
70.42
72.0
71.581
72.95
75.65100000000001
76.423
43.077
45.685
48.303000000000004
51.407
54.518
58.013999999999996
61.368
65.79899999999999
67.744
69.152
70.994
72.301
30.015
31.999000000000002
34.0
35.985
37.928000000000004
39.483000000000004
39.942
39.906
40.647
40.963
41.003
42.731
62.485
64.399
65.142
65.634
67.065
68.48100000000001
69.942
70.774
71.868
73.275
74.34
75.32
69.12
70.33
70.93
71.1
71.93
73.49
74.74
76.32
77.56
78.83
80.37
81.235
66.8
67.48
69.54
70.14
70.63
72.17
73.18
74.94
76.04
77.51
78.98
79.829
50.93899999999999
53.832
56.923
59.923
63.3
65.593
69.05199999999999
70.75
72.601
73.925
74.795
75.635
37.484
39.348
41.216
43.453
45.251999999999995
46.923
50.00899999999999
52.818999999999996
56.018
59.412
62.013000000000005
64.062
68.0
69.24
70.25
70.94
71.44
72.8
73.93
75.35
76.46
77.53
78.32
79.441
38.223
40.3580000

0    None
1    None
2    None
3    None
4    None
Name: lifeExp, dtype: object

In [11]:
# 평균값을 구하는 함수 만들기
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    return sum / n

In [12]:
# 사용자 정의 함수와 groupby 메서드를 조합하기 위해 agg 메서드 사용
agg_my_mean = df.groupby('year')['lifeExp'].agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [13]:
# 2개의 인잣값을 받아 처리하는 사용자함수 만들기
# 첫번쨰 인자로 받은 열의 평균값을 구하여, 두번째 인자로 받은 값과의 차이를 계산한 다음 반환하는 함수
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum +=  value
    mean = sum / n
    return mean - diff_value

In [14]:
# 연도별 평균 수명에서 전체 평균 수명을 뺀 값을 구함
global_mean = df['lifeExp'].mean()
print(global_mean)

agg_mean_diff = df.groupby('year')['lifeExp'].agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

59.47443936619713
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [15]:
# 여러개의 집계 메서드 한번에 사용하기 (함수 적용시 시리즈 형태로 출력되기 때문에 np사용)
gdf = df.groupby('year')['lifeExp'].agg([np.count_nonzero, np.mean,np.std])
print(gdf)


      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


## 11-2 데이터 변환
- 데이터 변환 메서드는 데이터와 메서드를 일대일로 대응시켜 계산하기 때문에 데이터 양은 줄어들지 않음

In [16]:
# 표준점수 계산하기 (데이터의 평균 - 표준편차)
def my_zscore(x):
    return (x-x.mean()) / x.std()

In [17]:
# transform 메서드 적용
transform_z = df.groupby('year')['lifeExp'].transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [18]:
# 누락값을 평균값으로 처리하기

# 똑같은 난수 생성을 원할때 사용
np.random.seed(42)

# tips 데이터 10개 가져오기
tips_10 = sns.load_dataset('tips').sample(10)

# total_bill 열의 값 4개를 임의로 선택하여 누락값으로 바꿈
tips_10.loc[np.random.permutation(tips_10.index)[:4],'total_bill'] = np.NaN
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [19]:
# 열의 누락값을 채울때 단순히 열의 평균값으로 채우면 안됨
# tips 데이터는 남성과 여성으로 나누어져있기 때문에 데이터를 살펴보고 해야함
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [20]:
# 성별을 구분하여 total_bill 열의 데이터를 받아 평균값을 구하는 함수
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg) #Nan값을 avg 값으로 바꿈

In [21]:
tips_10.groupby('sex')['total_bill'].mean()

sex
Male      17.9525
Female    13.9300
Name: total_bill, dtype: float64

In [22]:
# total_bill 열의 데이터를 함수에 전달하여 평균값을 구한 다음 새로운 열에 추가
# 남성의 Nan 값이 17.9525, 여성의 Nan 값이 13.93

total_bill_group_mean = tips_10.groupby('sex')['total_bill'].transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


## 11-3 데이터 필터링
- 그룹화한 데이터에서 원하는 데이터를 걸러내고 싶을때 사용

In [23]:
# 데이터 불러오기
tips = sns.load_dataset('tips')
print(tips.shape)
print(tips)

(244, 7)
     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


In [24]:
tips.groupby('size').count()
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [25]:
# 30번 이상의 주문이 있는 테이블만 그룹화하여 변수에 저장
tips_fillterd = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)
print(tips_fillterd)
print(tips_fillterd['size'].value_counts())

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[231 rows x 7 columns]
2    156
3     38
4     37
Name: size, dtype: int64


## 11-4 그룹 오브젝트

In [26]:
# 42번 seed에 있는 임의의 데이터 10개 불러오기
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [27]:
# groupby 메서드의 결괏값을 출력하면 자료형이 그룹 오브젝트라는 것을 확인
grouped = tips_10.groupby('sex')
print(grouped)

In [28]:
# 그룹 오브젝트에 포함된 그룹을 보려면 groups 속성을 출력
# 특정 열로 그룹화한 데이터프레임의 인덱스를 확인
print(grouped.groups)

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


In [29]:
# 한번에 그룹 오브젝트 계산하기
# 자동으로 계산할 수 있는 열을 골라줌
avgs = grouped.mean()
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [30]:
# 그룹 오브젝트에서 특정 데이터만 추출하기 (get_group)
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [31]:
# 각 성별의 그룹을 출력 => 반복문 사용
# 튜플의 형태로 값이 넘어옴

for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


### 여러 열을 사용해 그룹 오브젝트 만들고 계산하기

In [32]:
# 여러 열을 사용하여 데이터를 그룹화하려면 리스트에 열 이름을 담아 groupby 메서드에 전달
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()
print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [33]:
# 데이터 타입 확인하기
print(type(group_avg)) # 데이터프레임
print(group_avg.columns)
print(group_avg.index)

<class 'pandas.core.frame.DataFrame'>
Index(['total_bill', 'tip', 'size'], dtype='object')
MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [34]:
# 데이터프레임의 인덱스가 멀티 인덱스인 경우에는 reset_index를 사용하여 인덱스를 새로 부여할수 있음
group_method = tips_10.groupby(['sex','time']).mean().reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [35]:
# 데이터프레임의 인덱스가 멀티 인덱스인 경우에는 reset_index 대신 as_index = False를 사용해도 됨
group_method = tips_10.groupby(['sex','time'], as_index=False).mean()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
